# Deploy Model to VertextAI
- https://cloud.google.com/vertex-ai/docs/predictions/get-predictions#deploy_a_model_to_an_endpoint

## Load Model from Batch Prediction
- https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/get-batch-predictions
- https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/batch_prediction_model_monitoring.ipynb
- https://cloud.google.com/vertex-ai/docs/samples/aiplatform-create-batch-prediction-job-bigquery-sample#aiplatform_create_batch_prediction_job_bigquery_sample-python

In [ ]:
#chrome://bookmarks/?id=10824

# https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/batch_prediction_model_monitoring.ipynb

#https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/custom_batch_prediction_feature_filter.ipynb

# !gsutil cp gs://bp_mm_public_data/churn/churn_bp_outsample.jsonl  churn_bp_outsample.jsonl

#https://cloud.google.com/vertex-ai/docs/samples/aiplatform-create-batch-prediction-job-bigquery-sample#aiplatform_create_batch_prediction_job_bigquery_sample-python
# https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/get-batch-predictions
import numpy as np
import pandas as pd
from google.cloud import bigquery

from datetime import date, timedelta, datetime # Date Functions
import time

#import jsonlines
# with jsonlines.open('inoutput/incident_input_batch.jsonl', mode='w') as writer:
#   for data in listRequestData:  
#     writer.write(data)

In [ ]:
import random
import string
# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))
UUID = generate_uuid()
print(UUID)

In [ ]:
def load_data_bq(sql:str):
 client_bq = bigquery.Client()
 query_result=client_bq.query(sql)
 df_all=query_result.to_dataframe()
 return df_all

df=load_data_bq("SELECT * FROM `pongthorn.SMartML.new_incident`  LIMIT 5")
print(df.info())
df

# Create batch prediction job

In [ ]:
from google.cloud.aiplatform_v1beta1.types import (
    BatchDedicatedResources, BatchPredictionJob, GcsDestination, GcsSource,
    MachineSpec, ModelMonitoringAlertConfig, ModelMonitoringConfig,
    ModelMonitoringObjectiveConfig, ThresholdConfig)

In [ ]:
PROJECT_ID="pongthorn"
REGION="asia-southeast1"
MODEL_ID="xxxxxxxx"

TABLE_ID="pongthorn.SMartML.new_incident"

In [ ]:
BATCH_JOB_NAME = f"tf1_incident_batch_{UUID}"
MODEL_URI =  f'projects/{PROJECT_ID}/locations/{location}/models/{MODEL_ID}'
INPUT_FORMAT = "bigquery"

INPUT_URI = f"bq://{TABLE_ID}"
OUTPUT_FORMAT = "bigquery"
OUTPUT_URI = f"bq://{PROJECT_ID}"

MACHINE_TYPE = "n1-standard-2"


In [ ]:
EXCLUDED_FIELDS = ['id','severity_id','severity_name','imported_at']
INCLUDED_FIELDS= [ col  for col in df.columns.to_list() if col not in EXCLUDED_FIELDS ]
print(EXCLUDED_FIELDS)
print(INCLUDED_FIELDS)

# Create JSON body requests

In [ ]:

import json

request_with_excluded_fields = {
    "displayName": f"{BATCH_JOB_NAME}-excluded_fields",
    "model": MODEL_URI,
    "inputConfig": {
        "instancesFormat": INPUT_FORMAT,
        "bigquerySource": {"inputUri": INPUT_URI},
    },
    "outputConfig": {
        "predictionsFormat": OUTPUT_FORMAT,
        "bigqueryDestination": {"outputUri": OUTPUT_URI},
    },
    "dedicatedResources": {
        "machineSpec": {
            "machineType": MACHINE_TYPE,
        }
    },
    "instanceConfig": {"excludedFields": EXCLUDED_FIELDS},
}

with open("request_with_excluded_fields.json", "w") as outfile:
    json.dump(request_with_excluded_fields, outfile)
     

In [ ]:
request_with_included_fields = {
    "displayName": f"{BATCH_JOB_NAME}-included_fields",
    "model": MODEL_URI,
    "inputConfig": {
        "instancesFormat": INPUT_FORMAT,
        "bigquerySource": {"inputUri": INPUT_URI},
    },
    "outputConfig": {
        "predictionsFormat": OUTPUT_FORMAT,
        "bigqueryDestination": {"outputUri": OUTPUT_URI},
    },
    "dedicatedResources": {
        "machineSpec": {
            "machineType": MACHINE_TYPE,
        }
    },
    "instanceConfig": {"includedFields": INCLUDED_FIELDS},
}

with open("request_with_included_fields.json", "w") as outfile:
    json.dump(request_with_included_fields, outfile)

In [ ]:
# !gcloud auth application-default print-access-token
# ! curl \
#   -X POST \
#   -H "Authorization: Bearer $(gcloud auth application-default print-access-token)" \
#   -H "Content-Type: application/json" \
#   -d @request_with_excluded_fields.json \
#   https://{REGION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{REGION}/batchPredictionJobs

# Send the requests

In [ ]:
#https://cloud.google.com/vertex-ai/docs/samples/aiplatform-create-batch-prediction-job-bigquery-sample#aiplatform_create_batch_prediction_job_bigquery_sample-python

In [ ]:
parent = f"projects/{PROJECT_ID}/locations/{REGION}"
response = client.create_batch_prediction_job(
    parent=parent, batch_prediction_job=batch_prediction_job
)
print("response:", response)